Train MLP for all fingers!\
Using hyper param optimalization
Data source: \
sliding windowed powers for mu and beta band\
Subject 1

In [8]:
import os
import numpy as np
import h5py
from scipy import stats
import scipy.io
import mne

mne.set_log_level('error')

from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


import torch
import torch.nn as nn
import torch.optim as optim

import optuna


from utils.load import Load
from config.default import cfg

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
subject_id = 0

In [10]:
# Load the data  from the HDF5 file
target_dir = 'features'
file_path = os.path.join(target_dir, cfg['subjects'][subject_id] + '.h5')


data = {}
with h5py.File(file_path, 'r') as h5file:
    for key in h5file.keys():
        data[key] = np.array(h5file[key])

# Print the loaded data dictionary
for key, value in data.items():
    print(key, value.shape)

index (50, 6732)
little (50, 6732)
middle (50, 6732)
ring (50, 6732)
thumb (50, 6732)


In [32]:
class SingleLayerMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation):
        super(SingleLayerMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.activation = activation
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = nn.Softmax(dim=1)(x)
        return x

In [33]:
data.keys()

dict_keys(['index', 'little', 'middle', 'ring', 'thumb'])

In [34]:
def train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=100):
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)


    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        # # Shuffle X and y together
        p = np.random.permutation(len(X_train))
        X_train, y_train = X_train[p], y_train[p]

    with torch.no_grad():
        y_pred = model(X_test)
        y_pred = torch.argmax(y_pred, dim=1)

    acc = accuracy_score(y_test, y_pred)
    return acc

def objective(trial, X, y):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    num_epochs = trial.suggest_int("num_epochs", 100, 2000)
    hidden_size = trial.suggest_int("hidden_size", 16, 128)
    activation_name = trial.suggest_categorical("activation", ["relu", "elu", "leaky_relu"])
    optimizer = trial.suggest_categorical("optimizer", ["SGD", "Adam"])

    if activation_name == "relu":
        activation = nn.ReLU()
    elif activation_name == "elu":
        activation = nn.ELU()
    elif activation_name == "leaky_relu":
        activation = nn.LeakyReLU()

    if optimizer == "SGD":
        optimizer = optim.SGD
    elif optimizer == "Adam":
        optimizer = optim.Adam

    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
    model = SingleLayerMLP(train_X.shape[1], hidden_size, 5, activation)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    return train(train_X, train_y, test_X, test_y, model, criterion, optimizer, num_epochs=num_epochs)

    # kf = KFold(n_splits=10, shuffle=True, random_state=42)
    # fold_accuracies = []

    # for train_index, test_index in kf.split(X):
    #     X_train, X_test = X[train_index], X[test_index]
    #     y_train, y_test = y[train_index], y[test_index]

    #     model = SingleLayerMLP(X_train.shape[1], hidden_size, 2, activation)
    #     criterion = nn.CrossEntropyLoss()
    #     optimizer = optimizer(model.parameters(), lr=learning_rate)
    #     acc = train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=num_epochs)
    #     fold_accuracies.append(acc)

    # mean_accuracy = np.mean(fold_accuracies)
    # return mean_accuracy

def train_MLP(verbose = True):
    # Convert data into numpy arrays
    X = np.concatenate(list(data.values()), axis=0)
    y = np.concatenate([np.ones(data[finger].shape[0]) * i for i, finger in enumerate(data)], axis=0)

    # Shuffle X and y together
    # p = np.random.permutation(len(X))
    # X, y = X[p], y[p]

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=100)

    best_trial = study.best_trial

    print(f'Best trial params: {best_trial.params}')
    print(f'Best trial accuracy: {best_trial.value * 100:.2f}%')



In [35]:
train_MLP()

[I 2023-04-14 17:37:03,225] A new study created in memory with name: no-name-683cb7b0-cf6c-495c-8da9-e6dd95977dad
[I 2023-04-14 17:37:05,282] Trial 0 finished with value: 0.28 and parameters: {'learning_rate': 0.05398139258952086, 'num_epochs': 417, 'hidden_size': 19, 'activation': 'elu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.28.
[I 2023-04-14 17:37:11,322] Trial 1 finished with value: 0.24 and parameters: {'learning_rate': 8.170193615853764e-05, 'num_epochs': 1131, 'hidden_size': 58, 'activation': 'relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.28.
[I 2023-04-14 17:37:23,848] Trial 2 finished with value: 0.26 and parameters: {'learning_rate': 4.113958378658848e-05, 'num_epochs': 1270, 'hidden_size': 124, 'activation': 'leaky_relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.28.
[I 2023-04-14 17:37:28,701] Trial 3 finished with value: 0.16 and parameters: {'learning_rate': 5.6167326109395136e-05, 'num_epochs': 1452, 'hidden_size': 20, 'activation': 'rel

Best trial params: {'learning_rate': 0.008098366389804932, 'num_epochs': 1988, 'hidden_size': 103, 'activation': 'elu', 'optimizer': 'SGD'}
Best trial accuracy: 46.00%
